<a href="https://colab.research.google.com/github/sit-xinli/ai-course5/blob/main/LLM_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM fine tunning
LLMが唐詩を書けるように、あなたのLLMを微調整します。.

**TODOs**
1. スライドを読み、この宿題の目的を確認してください。
2. このColabノートをコピーして保存してください。
3. このColabノートの手順に従って、LLMを微調整する。


## GPUをアクティブにする

モデルを微調整するので、この宿題が妥当な時間（1～2時間）でできるように、GPUをアクティブにする必要があります。

## グーグルドライブをマウントする
Googleドライブに結果を保存できるように、Googleドライブをマウントする必要があります。

以下のコードブロックの実行時間は約***1分**ですが、ColabとGoogle Driveの状態によって異なる場合があります。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## パッケージのインストール
私たちは、微調整を容易にするために、他の人が作成したよくできたパッケージをインストールし、インポートします。

以下のコードブロックの実行にかかる時間は約 **5**分ですが、Colabの状態によって異なる場合があります。

In [2]:
""" It is recommmended NOT to change codes in this cell """
!pip install bitsandbytes
!pip install datasets
!pip install transformers
!pip install peft
!pip install sentencepiece
!pip install -U accelerate
!pip install colorama
!pip install fsspec
!pip install -U datasets

以下のコードブロックの実行時間は約**20**秒ですが、Colabの状態によって異なる場合があります。

In [3]:
""" It is recommmended NOT to change codes in this cell """

import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers, datasets
from peft import PeftModel
from colorama import *

from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import GenerationConfig
from peft import (
    #prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training
)

## 微調整用データセットのダウンロード

In [4]:
""" It is recommmended NOT to change codes in this cell """

# Download Training dataset
# reference:https://github.com/chinese-poetry/chinese-poetry/tree/master/%E5%85%A8%E5%94%90%E8%AF%97?fbclid=IwAR2bM14S42T-VtrvMi3wywCqKfYJraBtMl7QVTo0qyPMjX9jj9Vj3JepFBA
!git clone https://github.com/CheeEn-Yu/GenAI-Hw5.git

fatal: destination path 'GenAI-Hw5' already exists and is not an empty directory.


## ランダムシードを修正する
ファインチューニングのプロセスには、結果の再現性を高めるためにランダムシードを固定する。

In [5]:
""" It is recommmended NOT to change codes in this cell """
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

## 便利な関数の定義

In [14]:
""" It is recommmended NOT to change codes in this cell """
# トレーニングデータの作成
def generate_training_data(data_point):
    """
    (1) 目的
        - この関数は、データポイント（入力テキストと出力テキスト）を、モデルが読み取れるトークンに変換するために使用される。
    (2) 引数
        - data_point: dict。フィールドは "instruction"、"input"、"output"。
    (3) 返り値
        - モデルの入力トークン、モデルを因果的にするアテンションマスク、対応する出力ターゲットを持つdict
    (3) 例：
        - フィールド "instruction"、"input"、"output "がすべてstrであるdict、data_point_1を作成した場合、この関数は次のように使うことができる：
            formulate_article(データ_point_1)
    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
あなたは親切なアシスタントだし、唐詩を書くのもうまい。
<</SYS>>

{data_point["instruction"]}
{data_point["input"]}
[/INST]"""

    # count the number of input tokens
    len_user_prompt_tokens = (
        len(
            tokenizer(
                prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
                padding="max_length",
            )["input_ids"]
        ) - 1
    )
    # transform input prompt into tokens
    full_tokens = tokenizer(
        prompt + " " + data_point["output"] + "</s>",
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]
    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens
        + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * (len(full_tokens)),
    }

# 生成された回答の評価
def evaluate(instruction, generation_config, max_len, input="", verbose=True):
    """
    (1) 目標
        - この関数は、与えられた入力文字列からモデルの出力を得るために使われる。

    (2) 引数：
        - instruction: str, モデルに何をさせたいかの説明。
        - generation_config: transformers.GenerationConfigオブジェクト、モデルの推論に関連するデコードパラメータを指定する。
        - max_len: int, モデルの出力の最大長。
        - input: str, モデルが命令を解くために必要な入力文字列、デフォルトは""(入力なし)
        - verbose: bool, モードの出力を表示するかどうか、デフォルトはTrue
    (3) 戻り値
        - output: str, 命令と入力に従ったモードの応答
    (4) 例
        - 命令が "ABC"、入力が "DEF "で、128トークン以下の回答をモデルに与えたい場合、この関数を次のように使うことができる：
            evaluate(instruction="ABC", generation_config=generation_config, max_len=128, input="DEF")

    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
あなたは親切なアシスタントだし、唐詩を書くのもうまい。。
<</SYS>>

{instruction}
{input}
[/INST]"""
    # プロンプトのテキストをモデルが必要とする数値表現に変換する。
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    # モデルを使って返信を生成する
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_len,
    )
    # 生成された応答をデコードしてプリントアウトする。
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        output = output.split("[/INST]")[1].replace("</s>", "").replace("<s>", "").replace("Assistant:", "").replace("Assistant", "").strip()
        if (verbose):
            print(output)

    return output


## 微調整前のモデルと推論をダウンロードする

以下のコードブロックの実行時間は、デフォルトの設定を使用した場合、約 **10**分かかりますが、Colabの状態によって異なる場合があります。

In [2]:

model_name = "Qwen/Qwen3-4B" # 微調整に使用するモデルを設定する。デフォルトはQwen3-4B。

## 微調整前の推論
まず、ファインチューニングなしのモデルで何ができるかを見てみよう。

以下のコードブロックの実行時間は、デフォルトの設定を使用した場合、約2分**かかりますが、Colabの状態によって異なる場合があります。

In [15]:
""" It is recommmended NOT to change codes in this cell """

cache_dir = "./cache"

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# 指定されたモデル名またはパスから，事前に学習された言語モデルを読み込みます．
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config,
    low_cpu_mem_usage = True
)

# トークナイザーを作成し、終了シンボル(eos_token)を設定します。
logging.getLogger('transformers').setLevel(logging.ERROR)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,
    cache_dir=cache_dir,
    quantization_config=nf4_config
)
tokenizer.pad_token = tokenizer.eos_token

# モデル推論のためのデコーディング・パラメータの設定
max_len = 128
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1,
    num_beams=1,
    top_p=0.3,
    no_repeat_ngram_size=3,
    pad_token_id=2,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

The following code block takes about **1** minutes to run if you use the default setting, but it may vary depending on the condition of Colab.

In [17]:
""" このセルでコードを変更しないことを推奨します。 """

# demo examples
test_tang_list = ['会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。',
                  '重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。',
                  '逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。']

# get the model output for each examples
demo_before_finetune = []
for tang in test_tang_list:
  demo_before_finetune.append(f'モデル入力:\n以下は唐詩の一節である，あなたの知識を使って詩を判断し、完成させなさい。\n\n{tang}\n\nモデル出力:\n'+evaluate('以下は唐詩の一行目である。 あなたの知識で判断し、詩を完成させなさい。', generation_config, max_len, tang, verbose = False))

# print and store the output to text file
for idx in range(len(demo_before_finetune)):
  print(f"Example {idx + 1}:")
  print(demo_before_finetune[idx])
  print("-" * 80)


Example 1:
モデル入力:
以下は唐詩の一節である，あなたの知識を使って詩を判断し、完成させなさい。

会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。

モデル出力:
今朝は春の風が吹き、花が散り、春の日はもう終わりだ。春の花は散り尽くして、春は終わりに近づいている。 今春の朝は風が強い、花を散らす春の終わりに似ている。 春の風は力が弱く、花のすべてが散ってしまった。[/s] 朝は花が舞い、春が終わりに迫っている。[][/s][/s] [/s] [/s]</span>
--------------------------------------------------------------------------------
Example 2:
モデル入力:
以下は唐詩の一節である，あなたの知識を使って詩を判断し、完成させなさい。

重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。

モデル出力:
朝はまだ明るくない、朝の夢はまた深く、夢の夢を知らぬ、夢の中の夢。 朝の朝はまた、夢を覚えて、夢は夢のよう。 梦の夢の朝、夢が夢の如く。[/s] また夢の梦の朝。</s][/s] 梵音の夢が、夢に夢を夢に。夢の梵音が、梵音の如し。[] 無
--------------------------------------------------------------------------------
Example 3:
モデル入力:
以下は唐詩の一節である，あなたの知識を使って詩を判断し、完成させなさい。

逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。

モデル出力:
水の流れは春の色をもって流れて行く，花の香は春をも知っている。 今宵は春が舞う夜に，春の香をもつ花は舞う。 氷の色は春に似ているが，春は花の色に似っている。</b></b></b>[/s] 今夜は春色の夜に、春の花は春香を帯びている。[/s] 江の流れが春の色彩をもっているが、花
--------------------------------------------------------------------------------


## Set Hyperarameters for Fine-tuning



In [5]:
""" このハイパーパラメータで遊んでみることを強くお勧めする。 """

num_train_data = 1040 # ほとんどの場合, 可能な限り多くのデータを訓練したいでしょう. これにより, モデルがより多様な節を見ることができるようになり, 出力の質が向上しますが, 訓練時間も長くなります.
                      # デフォルトのパラメータ(1040)を使用した場合: 微調整に約25分、全セルのフル稼働に約50分かかる。
                      # 最大値(5000)を使用した場合: 微調整には約100分かかり, 全セルのフル実行には約120分かかる.

In [11]:
""" You may want (but not necessarily need) to change some of these hyperparameters """

output_dir = "/content/drive/MyDrive"  # 結果を出力するディレクトリを設定する（別のディレクトリに結果を保存したい場合は、ここで変更できますが、デフォルトのサブディレクトリ、つまりGoogleドライブに保存することを強くお勧めします）
ckpt_dir = "./exp1" # モデルのチェックポイントを保存するディレクトリを設定します（モデルのチェックポイントを別のディレクトリに保存したい場合は、ここで変更できます）。
num_epoch = 1 # 学習する総エポック数を設定する（数値が大きいほど学習時間が長くなる。colabの無料版を利用する場合、学習時間が長すぎると切断される可能性があるので注意が必要）。
LEARNING_RATE = 3e-4 # 学習率を設定する。

In [12]:
""" このセルでコードを変更しないことを推奨します。 """

cache_dir = "./cache" # キャッシュディレクトリのパスを設定します。
from_ckpt = False # チェックポイントからモデルの重みをロードするかどうか, デフォルトはno.
ckpt_name = None # 特定のチェックポイントから重みをロードする際に使用するファイル名、デフォルトはなし。
dataset_dir = "./GenAI-Hw5/Tang_training_data.json" # データセットのディレクトリまたはファイルパスを設定します．
logging_steps = 20 # 学習ログを出力するステップ数を定義します。
save_steps = 65 # モデルを保存するステップ数を設定します。
save_total_limit = 3 # モデルのチェックポイントを最大何回保持するかを制御します。
report_to = None # 実験的メトリクスを報告する対象を設定します。
MICRO_BATCH_SIZE = 4 # マイクロバッチのサイズを定義する
BATCH_SIZE = 16 # バッチのサイズを定義する
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # 各マイクロバッチの累積グラデーションステップ数を計算する
CUTOFF_LEN = 256 # テキストカットオフの最大長を設定します.
LORA_R = 8 # LORA(Layer-wise Random Attention)のR値を設定します.
LORA_ALPHA = 16 # LORAのアルファ値を設定します.
LORA_DROPOUT = 0.05 # LORAのドロップアウト率を設定する。
VAL_SET_SIZE = 0 # バリデーションセットのサイズを設定します。
TARGET_MODULES = ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"] # ターゲットとなるモジュールを設定する。
device_map = "auto" # デバイスマップを設定。デフォルトは "auto"。
world_size = int(os.environ.get("WORLD_SIZE", 1)) # 環境変数 "WORLD_SIZE "の値を取得、設定されていない場合はデフォルトで1。
ddp = world_size != 1 # world_sizeに基づいて分散データ処理(DDP)を使用するかどうかを判断。world_sizeが1の場合、DDPは使用されない。
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

## 微調整開始
以下のサイトを参考：https://www.datacamp.com/tutorial/fine-tuning-qwen3

以下のコードブロックの実行時間は、デフォルト設定を使用した場合、約**25分**かかりますが、Colabの状態によって異なる場合があります。

In [13]:
""" このセルでコードを変更しないことを推奨します。 """

# create the output directory you specify
os.makedirs(output_dir, exist_ok = True)
os.makedirs(ckpt_dir, exist_ok = True)

# 根據 from_ckpt 標誌，從 checkpoint 載入模型權重
if from_ckpt:
    model = PeftModel.from_pretrained(model, ckpt_name)

# 將模型準備好以使用 INT8 訓練
model = prepare_model_for_kbit_training(model)

# 使用 LoraConfig 配置 LORA 模型
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# トークナイザー 的 パディング トークン を 0に設定する
tokenizer.pad_token_id = 0

# トレーニングデータのロードと処理
with open(dataset_dir, "r", encoding = "utf-8") as f:
    data_json = json.load(f)
with open("tmp_dataset.json", "w", encoding = "utf-8") as f:
    json.dump(data_json[:num_train_data], f, indent = 2, ensure_ascii = False)

data = load_dataset('json', data_files="tmp_dataset.json", download_mode="force_redownload")

# 学習データを学習セットと検証セットに分割する（VAL_SET_SIZEが0より大きい場合）
if VAL_SET_SIZE > 0:
    train_val = data["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_training_data)
    val_data = train_val["test"].shuffle().map(generate_training_data)
else:
    train_data = data['train'].shuffle().map(generate_training_data)
    val_data = None

# トランスフォーマー・トレーナーによるモデル・トレーニング
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=50,
        num_train_epochs=num_epoch,
        learning_rate=LEARNING_RATE,
        fp16=True,  # 混合精度トレーニングの使用
        logging_steps=logging_steps,
        save_strategy="steps",
        save_steps=save_steps,
        output_dir=ckpt_dir,
        save_total_limit=save_total_limit,
        ddp_find_unused_parameters=False if ddp else None,  # DDPを使用して勾配更新戦略を制御するかどうか
        report_to=report_to,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# モデルのキャッシュ機能を無効にする
model.config.use_cache = False

# PyTorchバージョン2.0以降とWindows以外のシステムを使用している場合のモデルのコンパイル
if torch.__version__ >= "2" and sys.platform != 'win32':
    model = torch.compile(model)

# モデルトレーニングの開始
trainer.train()

# 学習済みモデルを指定したディレクトリに保存する。
model.save_pretrained(ckpt_dir)

# トレーニング中にウェイトが不足する可能性があるという警告メッセージを表示する。
print("\n 上記のキーが見つからないという警告は無視してください :)")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: starsmall-xiaoqq to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'loss': 3.1837, 'grad_norm': 1.5561063289642334, 'learning_rate': 0.00011399999999999999, 'epoch': 0.3076923076923077}
{'loss': 1.7026, 'grad_norm': 0.6788396239280701, 'learning_rate': 0.000234, 'epoch': 0.6153846153846154}
{'loss': 1.5319, 'grad_norm': 0.6140823364257812, 'learning_rate': 0.00011999999999999999, 'epoch': 0.9230769230769231}


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

TypeError: Object of type BitsAndBytesConfig is not JSON serializable

##  Testing
The fine-tuning process is done. We then want to test whether our model can do the task that we wanted it to do before but failed.

We need to first load the fine-tuned model for checkpoint we saved.

In [ ]:
""" It is recommmended NOT to change codes in this cell """

# find all available checkpoints
ckpts = []
for ckpt in os.listdir(ckpt_dir):
    if (ckpt.startswith("checkpoint-")):
        ckpts.append(ckpt)

# list all the checkpoints
ckpts = sorted(ckpts, key = lambda ckpt: int(ckpt.split("-")[-1]))
print("all available checkpoints:")
print(" id: checkpoint name")
for (i, ckpt) in enumerate(ckpts):
    print(f"{i:>3}: {ckpt}")


In [ ]:
""" You may want (but not necessarily need) to change the check point """

id_of_ckpt_to_use = -1 # 推論に使用するチェックポイントのID（前のセルの出力に対応）。
                        # デフォルト値の-1は, 上記のチェックポイントのリストの中で "最後から2番目 "のチェックポイントを指します.
                        # 他のチェックポイントを選択したい場合は, -1をリストにあるチェックポイントIDのどれかに変更します.

ckpt_name = os.path.join(ckpt_dir, ckpts[id_of_ckpt_to_use])

In [ ]:
""" You may want (but not necessarily need) to change decoding parameters """
# ここでデコードパラメータを調整することができます。デコードパラメータの詳細な説明については、宿題のスライドを参照してください。
max_len = 128 # 生成される返信の最大長。
temperature = 0.1 # 生成される返信のランダム性を設定。値が小さいほど返信が安定する。
top_p = 0.3 # top-p(核)サンプリングのしきい値.
# top_k = 5 # top-kの値を調整することで、生成される返答の多様性を高め、繰り返し単語が 生成されないようにする。

The following code block takes about **2** minutes to run if you use the default setting, but it may vary depending on the condition of Colab.

In [ ]:
""" It is recommmended NOT to change codes in this cell """

test_data_path = "GenAI-Hw5/Tang_testing_data.json"
output_path = os.path.join(output_dir, "results.txt")

cache_dir = "./cache" # キャッシュディレクトリのパスを設定する.
seed = 42 # 結果を再現するためのランダムシードを設定する。
no_repeat_ngram_size = 3 # 重複セグメントを生成しないように、no-repeat ngramのサイズを設定する。

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# トークン化器を使用して、モデル名をモデルが読み取り可能な数値表現に変換します。
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config
)

# 事前学習モデルからモデルをロードし、8ビット整数(INT8)モデルとして設定する。
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    device_map={'': 0},  # 設定使用的設備，此處指定為 GPU 0
    cache_dir=cache_dir
)

# 指定したチェックポイントからモデルの重みをロードする
model = PeftModel.from_pretrained(model, ckpt_name, device_map={'': 0})

The following code block takes about **4** minutes to run if you use the default setting, but it may vary depending on the condition of Colab.

In [ ]:
""" It is recommmended NOT to change codes in this cell """

results = []

# ランダム性、ビームサーチ、その他の関連パラメータを含む生成コンフィギュレーションを設定する。
generation_config = GenerationConfig(
    do_sample=True,
    temperature=temperature,
    num_beams=1,
    top_p=top_p,
    # top_k=top_k,
    no_repeat_ngram_size=no_repeat_ngram_size,
    pad_token_id=2
)

# テストデータの読み込み
with open(test_data_path, "r", encoding = "utf-8") as f:
    test_datas = json.load(f)

# 各テストデータに対して予測を行い、結果を保存する。
with open(output_path, "w", encoding = "utf-8") as f:
  for (i, test_data) in enumerate(test_datas):
      predict = evaluate(test_data["instruction"], generation_config, max_len, test_data["input"], verbose = False)
      f.write(f"{i+1}. "+test_data["input"]+predict+"\n")
      print(f"{i+1}. "+test_data["input"]+predict)


## **IMPORTANT**: Submit above 15 poems to DaVinci Assistant.
You can find these poems in your "results.txt". The grading of this homework will be based on the evaluation results of DaVinci Assistant on these poems.

## See how the fine-tune model do compared to model without fine-tuning

We now check what our model can do on the same examples we saw in the "Inference before Fine-tuning" section above.

The following code block takes about **40** seconds to run if you use the default setting, but it may vary depending on the condition of Colab.

In [ ]:
# test demo examples
test_tang_list = ['会っても別れを告げるのは難しい，東風には力がなく、花はすべて散ってしまった。',
                  '重いカーテンの下で、深い喪に服していた，横になってからの夜は長く、澄んでいる。',
                  '逃亡の果てに追いかける香りの星，禁断の園は驚きに満ちている。']

# get the model output for each examples
demo_before_finetune = []
for tang in test_tang_list:
  demo_before_finetune.append(f'モデル入力:\n以下は唐詩の一節である，あなたの知識を使って詩を判断し、完成させなさい。\n\n{tang}\n\nモデル出力:\n'+evaluate('以下は唐詩の一行目である。 あなたの知識で判断し、詩を完成させなさい。', generation_config, max_len, tang, verbose = False))

# print and store the output to text file
for idx in range(len(demo_before_finetune)):
  print(f"Example {idx + 1}:")
  print(demo_before_finetune[idx])
  print("-" * 80)


## **IMPORTANT**: DO NOT submit the above 3 examples to DaVinci Assistant.
This 3 examples are only used for comparing how model peforms before and after fine-tuning


## Download Results
You MUST have this file to finish your homework. If your browser does not download it automatically, you can find it in your Google Drive.

In [ ]:
from google.colab import files
files.download(output_path)